In [23]:
from modelscope.models import Model
from modelscope.pipelines import pipeline
from modelscope.preprocessors import ConversationalTextToSqlPreprocessor
from modelscope.utils.constant import Tasks

In [43]:
import sqlite3

conn = sqlite3.connect('C:/Users/Cherw/.cache/modelscope/hub/damo/nlp_star_conversational-text-to-sql/db/employee_hire_evaluation/employee_hire_evaluation.sqlite')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

[('employee',), ('shop',), ('hiring',), ('evaluation',)]

In [44]:
import pandas as pd

from IPython.display import display, HTML

for table_name in tables:
    table_name = table_name[0]
    table = pd.read_sql_query("SELECT * from %s" % table_name, conn)
    display (table)

,Employee_ID,Name,Age,City
0,1,George Chuter,23,Bristol
1,2,Lee Mears,29,Bath
2,3,Mark Regan,43,Bristol
3,4,Jason Hobson,30,Bristol
4,5,Tim Payne,29,Wasps
5,6,Andrew Sheridan,28,Sale
6,7,Matt Stevens,29,Bath
7,8,Phil Vickery,40,Wasps
8,9,Steve Borthwick,32,Bath
9,10,Louis Deacon,36,Leicester


,Shop_ID,Name,Location,District,Number_products,Manager_name
0,1,FC Haka,Valkeakoski,Tehtaan kenttä,3516,Olli Huttunen
1,2,HJK,Helsinki,Finnair Stadium,10770,Antti Muurinen
2,3,FC Honka,Espoo,Tapiolan Urheilupuisto,6000,Mika Lehkosuo
3,4,FC Inter,Turku,Veritas Stadion,10000,Job Dragtsma
4,5,FF Jaro,Jakobstad,Jakobstads Centralplan,5000,Mika Laurikainen
5,6,FC KooTeePee,Kotka,Arto Tolsa Areena,4780,Tommi Kautonen
6,7,KuPS,Kuopio,Magnum Areena,3500,Kai Nyyssönen
7,8,FC Lahti,Lahti,Lahden Stadion,15000,Ilkka Mäkelä
8,9,IFK Mariehamn,Mariehamn,Wiklöf Holding Arena,1600,Pekka Lyyski


,Shop_ID,Employee_ID,Start_from,Is_full_time
0,1,1,2009,T
1,1,2,2003,T
2,8,3,2011,F
3,4,4,2012,T
4,5,5,2013,T
5,2,6,2010,F
6,6,7,2008,T


,Employee_ID,Year_awarded,Bonus
0,1,2011,3000.0
1,2,2015,3200.0
2,1,2016,2900.0
3,4,2017,3200.0
4,7,2018,3200.0
5,10,2016,4000.0


In [24]:
model_id = 'damo/nlp_star_conversational-text-to-sql'
test_case = {
    "database_id": 'employee_hire_evaluation',
    'local_db_path':None,
    "utterance":[
    "I'd like to see Shop names.",
    "Which of these are hiring?",
    "Which shop is hiring the highest number of employees? | do you want the name of the shop ? | Yes"]
}

In [25]:
model = Model.from_pretrained(model_id)
preprocessor = ConversationalTextToSqlPreprocessor(model_dir=model.model_dir)
pipeline = pipeline(task=Tasks.table_question_answering,
                    model=model,
                    preprocessor=preprocessor)

2023-05-03 19:41:36,365 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
2023-05-03 19:41:38,072 - modelscope - INFO - initialize model from C:\Users\Cherw\.cache\modelscope\hub\damo\nlp_star_conversational-text-to-sql
2023-05-03 19:41:49 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2023-05-03 19:41:49 INFO: Using device: cpu
2023-05-03 19:41:49 INFO: Loading: tokenize
2023-05-03 19:41:49 INFO: Loading: pos
2023-05-03 19:41:50 INFO: Loading: lemma
2023-05-03 19:41:50 INFO: Done loading processors!
2023-05-03 19:41:50 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2023-05-03 19:41:50 INFO: Using device: cpu
2023-05-03 19:41:50 INFO: Loading: tokenize
2023-05-03 19:41:50 INFO: Loading: po

In total, process 1 databases .


In [26]:
last_sql = ''
history = []
for item in test_case['utterance']:
    case = {"utterance": item,
            "history": history,
            "last_sql": last_sql,
            "database_id": test_case['database_id'],
            'local_db_path': test_case['local_db_path']}
    results = pipeline(case)
    print(results)
    history.append(item)

punkt has already exist!
{'output': {'text': 'SELECT shop.Name FROM shop'}}
punkt has already exist!
{'output': {'text': 'SELECT shop.Name FROM shop'}}
punkt has already exist!
{'output': {'text': 'SELECT shop.Name FROM hiring JOIN shop GROUP BY hiring.Shop_ID ORDER BY COUNT(*) DESC LIMIT 1'}}


In [45]:
test_case = {
    "database_id": 'employee_hire_evaluation',
    'local_db_path':None,
    "utterance":[
    "I'd like to see Shop names.",
    "Which employee had highest bonus?",
    "Which shop is hiring the highest number of employees? | do you want the name of the shop ? | Yes"]
}

In [46]:
last_sql = ''
history = []
for item in test_case['utterance']:
    case = {"utterance": item,
            "history": history,
            "last_sql": last_sql,
            "database_id": test_case['database_id'],
            'local_db_path': test_case['local_db_path']}
    results = pipeline(case)
    print(results)
    history.append(item)

punkt has already exist!
{'output': {'text': 'SELECT shop.Name FROM shop'}}
punkt has already exist!
{'output': {'text': 'SELECT evaluation.Employee_ID FROM evaluation ORDER BY evaluation.Bonus DESC LIMIT 1'}}
punkt has already exist!
{'output': {'text': 'SELECT shop.Name FROM hiring JOIN shop GROUP BY hiring.Shop_ID ORDER BY COUNT(*) DESC LIMIT 1'}}
